In [0]:
sp_client_secret = dbutils.secrets.get(
    scope="kv-etl-scope", 
    key="spn-etl-gold-sql-secret"
)

print("Retrieved the service principal secret securely!")

spark.conf.set(
  "fs.azure.account.key.stetlprojectdeveus.dfs.core.windows.net",
  dbutils.secrets.get(scope="kv-etl-scope", key="storage-account-key")
)

In [0]:
sp_client_id = dbutils.secrets.get(scope="kv-etl-scope", key="spn-etl-gold-sql-clientid")
tenant_id = dbutils.secrets.get(scope="kv-etl-scope", key="spn-etl-gold-sql-tenantid")
sp_client_secret = dbutils.secrets.get(scope="kv-etl-scope", key="spn-etl-gold-sql-secret")

jdbc_server = "sqlsv-etldatabricks-dev-eastus2.database.windows.net"
jdbc_database = "sqldb-etldatabricks-dev"


In [0]:
jdbc_port = "1433"
jdbc_url = f"""jdbc:sqlserver://{jdbc_server}:{jdbc_port};database={jdbc_database};
encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"""


In [0]:
silver_input_path = "abfss://datalake@stetlprojectdeveus.dfs.core.windows.net/silver/"

df_silver = spark.read.parquet(silver_input_path)

print(f"Silver row count: {df_silver.count()}")
df_silver.printSchema()
display(df_silver.limit(10))


In [0]:
from pyspark.sql import functions as F

df_gold = (
    df_silver
    .groupBy("Department")
    .agg(
        F.count("*").alias("Total_Employees"),
        F.round(F.avg("Monthly_Salary"), 2).alias("Avg_Monthly_Salary"),
        F.round(F.avg("Employee_Satisfaction_Score"), 2).alias("Avg_Satisfaction"),
        F.round(F.avg("Performance_Score"), 2).alias("Avg_Performance"),
        F.sum(F.when(F.col("HasResigned") == True, 1).otherwise(0)).alias("Resigned_Count")
    )
    .orderBy("Department")
)


In [0]:
display(df_gold)


In [0]:
df_gold.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.Gold_DepartmentSummary") \
    .option("user", sp_client_id) \
    .option("password", sp_client_secret) \
    .option("authentication", "ActiveDirectoryServicePrincipal") \
    .save()


In [0]:
from pyspark.sql.functions import avg, count, round, sum, col

df_gold_kpis = df_silver.groupBy("Department").agg(
    count("*").alias("Employee_Count"),
    round(avg("Monthly_Salary"), 2).alias("Avg_Salary"),
    round(avg("Employee_Satisfaction_Score"), 2).alias("Avg_Satisfaction"),
    sum(col("HasResigned").cast("int")).alias("Resignations")
)


In [0]:
df_gold_top_promotions = df_silver.groupBy("Department") \
    .agg(sum("Promotions").alias("Total_Promotions")) \
    .orderBy("Total_Promotions", ascending=False)


In [0]:
df_gold_edu_salary = df_silver.groupBy("Education_Level") \
    .agg(round(avg("Monthly_Salary"), 2).alias("Avg_Salary")) \
    .orderBy("Avg_Salary", ascending=False)


In [0]:
df_gold_work_satisfaction = df_silver.select(
    "Department", "Work_Hours_Per_Week", "Overtime_Hours", "Employee_Satisfaction_Score"
)


In [0]:
from pyspark.sql.functions import sum, col

df_gold_resign_age = df_silver.groupBy("Age") \
    .agg(
        sum(col("HasResigned").cast("int")).alias("Total_Resignations")
    ) \
    .orderBy("Age")


In [0]:
from pyspark.sql.functions import count, round

df_gold_resign_rate_age = df_silver.groupBy("Age").agg(
    count("*").alias("Total_Employees"),
    sum(col("HasResigned").cast("int")).alias("Total_Resignations"),
    round(
        (sum(col("HasResigned").cast("int")) / count("*")) * 100, 2
    ).alias("Resignation_Rate_Percentage")
).orderBy("Age")


In [0]:
df_gold_kpis.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.Gold_DepartmentKPI") \
    .option("user", sp_client_id) \
    .option("password", sp_client_secret) \
    .option("authentication", "ActiveDirectoryServicePrincipal") \
    .save()


In [0]:
df_gold_resign_age.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.Gold_ResignationsByAge") \
    .option("user", sp_client_id) \
    .option("password", sp_client_secret) \
    .option("authentication", "ActiveDirectoryServicePrincipal") \
    .save()


In [0]:
df_gold_work_satisfaction.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.Gold_WorkSatisfaction") \
    .option("user", sp_client_id) \
    .option("password", sp_client_secret) \
    .option("authentication", "ActiveDirectoryServicePrincipal") \
    .save()


In [0]:
df_gold_edu_salary.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.Gold_EducationSalary") \
    .option("user", sp_client_id) \
    .option("password", sp_client_secret) \
    .option("authentication", "ActiveDirectoryServicePrincipal") \
    .save()


In [0]:
df_gold_top_promotions.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.Gold_TopDepartments_Promotions") \
    .option("user", sp_client_id) \
    .option("password", sp_client_secret) \
    .option("authentication", "ActiveDirectoryServicePrincipal") \
    .save()


In [0]:
df_gold_top_promotions.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.Gold_TopDepartments_Promotions") \
    .option("user", sp_client_id) \
    .option("password", sp_client_secret) \
    .option("authentication", "ActiveDirectoryServicePrincipal") \
    .save()
